# Manual Relation/Alignment Prototype in Jupyter NB
Purpose: Explore options for inputing relations

In [ ]:
# %pip install ipympl
# %pip install pyqt5

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath("../.."))  # Repository directory relative to this file.
from MISalign.model.project import Project
from MISalign.model.relation import Relation
from MISalign.model.relation_service import relation_tree
from MISalign.model.image import Image

from PIL import Image as PILImage

from matplotlib import pyplot as plt

In [3]:
from os.path import abspath

In [ ]:
%matplotlib qt5
img=PILImage.open(abspath(r"..\..\example\data\set_a\a_myimages01.jpg"))
# display(img)
plt.imshow(img)
plt.ginput(5)


In [7]:
%matplotlib qt5
img0=PILImage.open(abspath(r"..\..\example\data\set_a\a_myimages01.jpg"))
img1=PILImage.open(abspath(r"..\..\example\data\set_a\a_myimages02.jpg"))

fig0, ax0 = plt.subplots()
ax0.imshow(img0)
plt.title("Figure 0")

fig1, ax1 = plt.subplots()
ax1.imshow(img1)
plt.title("Figure 1")
plt.show()

In [53]:
plt.autoscale(tight=True)
# select up to 3 points using left mouse
# stop collecting points with middle mouse
# undo previous with right click
border=ax0.plot([0,0,1600,1600,0],[0,1200,1200,0,0],"r--")
# plt.show()
fig0.draw(renderer=fig0.canvas.renderer)
plt.pause(0.05)
# plt.pause(5)
# fig0.show()
# fig0.canvas.flush_events()
fg0=fig0.ginput(3)
ax0.lines[0].remove()
fig0.draw(renderer=fig0.canvas.renderer)
plt.pause(0.05)

border=ax1.plot([0,0,1600,1600,0],[0,1200,1200,0,0],"r--")
# fig0.show()
# fig1.show()
# fig0.canvas.flush_events()
# fig1.canvas.flush_events()
# plt.show()
# plt.pause(5)
fig1.draw(renderer=fig0.canvas.renderer)
plt.pause(0.05)
fg1=fig1.ginput(3)
ax1.lines[0].remove()
fig1.draw(renderer=fig0.canvas.renderer)
plt.pause(0.05)
print(fg0,fg1)
# fig1.show()
plt.show()

[(795.0085582620145, 704.4448650427914), (850.5648452929559, 802.1473008558263), (971.2560895325871, 618.2368334430547)] [(636.002633311389, 340.4553982883476), (967.4246214614877, 396.011685319289), (1114.9361421988149, 307.8879196840028)]


In [12]:
ax0.imshow(img1)
ax0.set_axis_off()
img2=PILImage.open(abspath(r"..\..\example\data\set_a\a_myimages03.jpg"))
ax1.imshow(img2)
ax1.set_axis_off()
plt.show()